In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# installing the packages
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu1

In [ ]:
# loading dataset
from datasets import load_dataset

dataset = load_dataset("persiannlp/parsinlu_entailment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1675 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/270 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 755
    })
    test: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 1675
    })
    validation: Dataset({
        features: ['sent1', 'sent2', 'category', 'label'],
        num_rows: 270
    })
})

In [ ]:
# Define label mapping
label_map = {"e": 0, "c": 1, "n": 2}

# Filter out examples with label 'xx'
def filter_function(examples):
    return examples['label'] in label_map.keys()

# Apply the filter function to both train and validation datasets
filtered_train_dataset = dataset['train'].filter(filter_function)
filtered_validation_dataset = dataset['validation'].filter(filter_function)

Filter:   0%|          | 0/755 [00:00<?, ? examples/s]

Filter:   0%|          | 0/270 [00:00<?, ? examples/s]

In [ ]:
dataset['train'][0]

{'sent1': 'زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد.',
 'sent2': 'مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.',
 'category': 'translation-train',
 'label': 'c'}

In [ ]:
# Encode labels
def encode_labels(examples):
    examples['label'] = label_map[examples['label']]
    return examples

filtered_train_dataset = filtered_train_dataset.map(encode_labels)
filtered_validation_dataset = filtered_validation_dataset.map(encode_labels)

Map:   0%|          | 0/754 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [ ]:
filtered_train_dataset[0]

{'sent1': 'زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد.',
 'sent2': 'مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.',
 'category': 'translation-train',
 'label': 1}

In [ ]:
set(filtered_train_dataset['label'])

{0, 1, 2}

In [ ]:
# downloading the model and tokenizer
from transformers import AutoTokenizer

model_name_or_path = "persiannlp/parsbert-base-parsinlu-entailment"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# here I tokenize sent1 and sent2 and have int label
def tokenize_function(examples):
    tokenized = tokenizer(examples['sent1'], examples['sent2'], padding="max_length", truncation=True, max_length=128)
    tokenized['labels'] = examples['label']
    return tokenized


tokenized_train_dataset = filtered_train_dataset.map(tokenize_function, batched=False, remove_columns=['sent1', 'sent2', 'category','label'])
tokenized_validation_dataset = filtered_validation_dataset.map(tokenize_function, batched=False, remove_columns=['sent1', 'sent2', 'category','label'])

tokenizer_config.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/754 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 754
})

In [ ]:
tokenized_train_dataset[0]

{'input_ids': [2,
  4039,
  2789,
  11053,
  3108,
  5020,
  2791,
  4290,
  2867,
  2803,
  3630,
  2793,
  4577,
  2800,
  2789,
  6192,
  2793,
  3435,
  4686,
  2996,
  2800,
  3064,
  2802,
  4389,
  2786,
  3050,
  4039,
  6748,
  6198,
  1348,
  2802,
  4389,
  2793,
  25408,
  6748,
  3048,
  1012,
  4,
  5590,
  3108,
  10542,
  2791,
  4290,
  2867,
  3060,
  3882,
  3327,
  3440,
  3337,
  3060,
  1012,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [ ]:
# LORA configuration object using Hugging Face’s Efficient Fine-Tuning (PEFT) parameters.
# The task_Type parameter specifies the task type for which the model will undergo fine-tuning.
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [ ]:
from transformers import BertForSequenceClassification
# Load a pre-trained BERT model for sequence classification with 3 labels
model = BertForSequenceClassification.from_pretrained(
    model_name_or_path,
    num_labels=3
)

pytorch_model.bin:   0%|          | 0.00/651M [00:00<?, ?B/s]

In [ ]:
from peft import get_peft_model
# Apply the PEFT (Parameter-Efficient Fine-Tuning) model with LoRA configuration to the pre-trained model
model = get_peft_model(model, lora_config)

In [ ]:
import numpy as np
import evaluate
# Load the accuracy metric for evaluation
metric = evaluate.load("accuracy")

In [ ]:
# Define a function to compute evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
# Set up training arguments, including output directory, evaluation strategy, and number of epochs
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/entailment/test_trainer1", evaluation_strategy="epoch",
                                 num_train_epochs=25,)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.489495,0.544444
2,No log,1.499819,0.544444
3,No log,1.525586,0.537037
4,No log,1.554343,0.533333
5,No log,1.560405,0.533333
6,0.353700,1.556737,0.533333
7,0.353700,1.570258,0.533333
8,0.353700,1.592099,0.525926
9,0.353700,1.583691,0.525926
10,0.353700,1.573271,0.529630


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=2375, training_loss=0.33124230635793583, metrics={'train_runtime': 347.4718, 'train_samples_per_second': 54.249, 'train_steps_per_second': 6.835, 'total_flos': 1240489051776000.0, 'train_loss': 0.33124230635793583, 'epoch': 25.0})

In [ ]:
# Save the trained model and tokenizer to the specified directory
model.save_pretrained("/content/drive/MyDrive/entailment/results1")
tokenizer.save_pretrained("/content/drive/MyDrive/entailment/results1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/entailment/results1/tokenizer_config.json',
 '/content/drive/MyDrive/entailment/results1/special_tokens_map.json',
 '/content/drive/MyDrive/entailment/results1/vocab.txt',
 '/content/drive/MyDrive/entailment/results1/added_tokens.json',
 '/content/drive/MyDrive/entailment/results1/tokenizer.json')

# inference

In [ ]:
import torch
from peft import PeftModel
from transformers import  AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM


In [ ]:
# Load the entire model on the GPU 0
device_map = {"": 0}
# fine-tuned model path
path = "/content/drive/MyDrive/entailment/results1"

# base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
base_model = BertForSequenceClassification.from_pretrained(model_name_or_path)
model = PeftModel.from_pretrained(base_model, path)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

In [ ]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(100000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (defaul

In [ ]:
# Import necessary modules from torch and transformers libraries
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np

# Define labels for classification
labels = ["entails", "contradicts", "neutral"]
# Specify the model name or path for the pre-trained model
model_name_or_path = "persiannlp/parsbert-base-parsinlu-entailment"
# Load the pre-trained model and tokenizer for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,)

# Define a function to make predictions using the model
def model_predict(text_a, text_b):
    features = tokenizer( [(text_a, text_b)], padding="max_length", truncation=True, return_tensors='pt')
    output = model(**features)
    logits = output[0]
    probs = torch.nn.functional.softmax(logits, dim=1).tolist()
    idx = np.argmax(np.array(probs))
    print(labels[idx], probs)


model_predict(
    "این مسابقات بین آوریل و دسامبر در هیپودروم ولیفندی در نزدیکی باکرکی ، ۱۵ کیلومتری (۹ مایل) غرب استانبول برگزار می شود.",
    "در ولیفندی هیپودروم، مسابقاتی از آوریل تا دسامبر وجود دارد."
)

model_predict(
"آیا کودکانی وجود دارند که نیاز به سرگرمی دارند؟",
    "هیچ کودکی هرگز نمی خواهد سرگرم شود.",
)

model_predict(
    "ما به سفرهایی رفته ایم که در نهرهایی شنا کرده ایم",
    "علاوه بر استحمام در نهرها ، ما به اسپا ها و سونا ها نیز رفته ایم."
)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


entails [[0.9939858317375183, 0.004851127974689007, 0.0011631686938926578]]
contradicts [[0.060658086091279984, 0.9340943098068237, 0.005247595254331827]]
neutral [[0.0019120053621008992, 0.000729170220438391, 0.9973588585853577]]


In [ ]:
labels = ["e", "c", "n"]
model_predict(
    "زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد",
    "مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند"
)

c [[0.0014494083588942885, 0.9965196847915649, 0.002030889503657818]]


In [ ]:
labels = ["e", "c", "n"]
model_predict(
    "زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد",
    "مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند"
)

c [[0.0014494083588942885, 0.9965196847915649, 0.002030889503657818]]
